# Compass LangChain Toolkit

The `langchain-compass` toolkit contains tools that enable an LLM agent to interact with popular DeFi protocols non-custodially. The toolkit is built on top of a Universal DeFi API (Compass API) allowing agents to perform financial operations like:

- **Swapping tokens** on Uniswap and Aerodrome
- **Lending** or **borrowing** assets using protocols on Aave
- **Providing liquidity** on Aerodrome and Uniswap
- **Transferring funds** between wallets.
- Querying balances, portfolios and **monitoring positions**.

## Overview

### Integration details

| Class                    | Package             | Serializable | JS support |                                         Package latest                                         |
|:-------------------------|:--------------------| :---: | :---: |:----------------------------------------------------------------------------------------------:|
| LangchainCompassToolkit  | `langchain-compass` | ❌ | ❌ | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-compass?style=flat-square&label=%20) |

### Tool features

Here’s a sample of the tools this toolkit provides (subject to change daily):
	- 	`aave_supply_`: Supply assets to Aave to earn interest.
	- 	`aave_borrow_`: Borrow assets from Aave using collateral.
	- 	`uniswap_swap_sell_exactly_`: Swap a specific amount of one token on Uniswap.
	- 	`generic_portfolio_get_`: Retrieve a wallet’s portfolio in USD and token balances.
	- 	`generic_transfer_erc20_`: Transfer ERC20 tokens to another address.


## Setup

At a high-level, we will:

1. Install the langchain package
2. Import and instantiate the toolkit
3. Pass the tools to your agent with `toolkit.get_tools()`

To enable automated tracing of individual tools, set your [LangSmith](https://docs.smith.langchain.com/) API key:

In [1]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

This toolkit lives in the `langchain-compass` package:

In [2]:
%pip install -qU langchain-compass

/Users/johncosnett/compass_labs/compass_ai/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


#### Environment Setup

To run these examples, ensure LangChain has access to an LLM service. For instance, if you're using GPT-4o, create a `.env` file containing:

In [3]:
# OPENAI_API_KEY=your_openai_api_key_here

## Instantiation

Now we can instantiate our toolkit:

In [4]:
from langchain_compass.toolkits import LangchainCompassToolkit

toolkit = LangchainCompassToolkit(compass_api_key=None)

## Tools

View [available tools](#tool-features):

In [5]:
tools = toolkit.get_tools()
for tool in tools:
    print(tool.name)

aave_supply_
aave_borrow_
aave_repay_
aave_withdraw_
aave_asset_price_get_
aave_liquidity_change_get_
aave_user_position_summary_get_
aave_user_position_per_token_get_
aerodrome_slipstream_swap_sell_exactly_
aerodrome_slipstream_swap_buy_exactly_
aerodrome_slipstream_liquidity_provision_mint_
aerodrome_slipstream_liquidity_provision_increase_
aerodrome_slipstream_liquidity_provision_withdraw_
aerodrome_slipstream_liquidity_provision_positions_get_
aerodrome_slipstream_pool_price_get_
generic_portfolio_get_
generic_visualize_portfolio_get_
generic_price_usd_get_
generic_supported_tokens_get_
generic_balance_get_
generic_allowance_get_
generic_ens_get_
generic_wrap_eth_
generic_unwrap_weth_
generic_transfer_erc20_
generic_transfer_native_token_
generic_allowance_set_
uniswap_swap_buy_exactly_
uniswap_swap_sell_exactly_
uniswap_liquidity_provision_increase_
uniswap_liquidity_provision_mint_
uniswap_liquidity_provision_withdraw_
uniswap_quote_buy_exactly_get_
uniswap_quote_sell_exactly_get

## Use within an agent

We will need a LLM or chat model:

In [6]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model="gpt-4o")

Initialize the agent with the tools:

In [7]:
from langgraph.prebuilt import create_react_agent

tools = toolkit.get_tools()
agent_executor = create_react_agent(llm, tools)

Example usage:

In [8]:
example_query = "what is the balance of vitalic.eth."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what is the balance of vitalic.eth.
================================== Ai Message ==================================
Tool Calls:
  generic_ens_get_ (call_kbgxVmdMNzaJa79srcI9wgTf)
 Call ID: call_kbgxVmdMNzaJa79srcI9wgTf
  Args:
    chain: ethereum:mainnet
    ens_name: vitalik.eth
================================= Tool Message =================================
Name: generic_ens_get_

wallet_address='0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045' registrant='0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045'
================================== Ai Message ==================================
Tool Calls:
  generic_portfolio_get_ (call_s5BMj4zrqy3ir1kmfxh7TQB7)
 Call ID: call_s5BMj4zrqy3ir1kmfxh7TQB7
  Args:
    chain: ethereum:mainnet
    user: 0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045
================================= Tool Message =================================
Name: generic_portfolio_get_

total_value_in_usd='48551

Expected output:
```
The portfolio value of the wallet address associated with `vitalik.eth` is approximately $485,515.45 USD. Below is a detailed breakdown of the token balances:

1. **1INCH**: `6.04` tokens valued at approximately `$1.02` USD
2. **AAVE**: `0.0102` tokens valued at approximately `$1.40` USD
3. **BAL**: `0.9321` tokens valued at approximately `$0.96` USD
4. **crvUSD**: `0.7755` tokens valued at approximately `$0.78` USD
5. **DAI**: `317,203.87` tokens valued at approximately `$317,292.76` USD
6. **ENS**: `1,144.04` tokens valued at approximately `$16,329.73` USD
7. **LINK**: `1.78` tokens valued at approximately `$21.95` USD
8. **rsETH**: `0.00003` tokens valued at approximately `$0.05` USD
9. **UNI**: `0.000017` tokens valued at approximately `$0.00009` USD
10. **USDC**: `123,223.71` tokens valued at approximately `$123,215.08` USD
11. **USDT**: `170.15` tokens valued at approximately `$170.12` USD
12. **WBTC**: `0.00107` tokens valued at approximately `$90.97` USD
13. **WETH**: `16.40` tokens valued at approximately `$28,390.62` USD

The total value from all these assets comes to approximately $485,515.45 USD.
```